In [1]:
# CEP OCCASSIONAL PAPER CHARTS
# Will Shepherd, Nov 2025

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import altair as alt
from pandas.api.types import CategoricalDtype
import os
import eco_style 
alt.themes.enable("report")

ThemeRegistry.enable('report')

In [2]:
# Import data
whole_economy_df = pd.read_excel('business_dynamism_BSD_1997_2023.xlsx', sheet_name='whole_economy')
firm_size_df = pd.read_excel('business_dynamism_BSD_1997_2023.xlsx', sheet_name='firm_size')
firm_age_df = pd.read_excel('business_dynamism_BSD_1997_2023.xlsx', sheet_name='firm_age')
industry_df = pd.read_excel('business_dynamism_BSD_1997_2023.xlsx', sheet_name='industry')
region_df = pd.read_excel('business_dynamism_BSD_1997_2023.xlsx', sheet_name='region')

In [3]:
# Define order for categorical variables

# 1. Employment sizeband
size_order = [
    'Large (250+)'
    'Medium (50-249)'
    'Small (10-49)'
    'Micro (0-9)'
]

# 2. Age group
age_order = [
    'New (0-2 years)',
    'Young (3-5 years)',
    'Old (5-10 years)',
    'Mature (10+ years)'
]

In [4]:
# Write function to calculate rates for dynamism measures, apply this across dataframes
def calculate_dynamism_rates(df, group_by_cols=None):
    # Make a copy to avoid modifying the original
    df = df.copy()
    
    # Sort data
    sort_cols = group_by_cols + ['year'] if group_by_cols else ['year']
    df = df.sort_values(sort_cols)
    
    # Create lagged employment (with or without grouping)
    if group_by_cols is None:
        df['total_employment_lagged'] = df['employment'].shift(1)
    else:
        df['total_employment_lagged'] = df.groupby(group_by_cols)['employment'].shift(1)
    
    # Calculate rates (same regardless of grouping)
    df['Entry rate'] = (df['n_entrants'] + df['n_entry_and_exit']) / df['n_firms']
    df['Exit rate'] = (df['n_exiters'] + df['n_entry_and_exit']) / df['n_firms']
    df['Job creation rate'] = (df['jc_incumbents'] + df['jc_entrants']) / df['total_employment_lagged']
    df['Job destruction rate'] = (df['jd_incumbents'] + df['jd_exiters']) / df['total_employment_lagged']
    df['Entry job creation rate'] = (df['jc_entrants']) / df['total_employment_lagged']
    df['Incumbent job creation rate'] = (df['jc_incumbents']) / df['total_employment_lagged']
    df['Exit job destruction rate'] = (df['jd_exiters']) / df['total_employment_lagged']
    df['Incumbent job destruction rate'] = (df['jd_incumbents']) / df['total_employment_lagged']


    # We can't use the first/last year for dynamic variables due to no backward/forward looking observatinons
    years = df['year'].unique()
    df = df[~df['year'].isin([years.min(), years.max()])]

    return df

# Apply function to dataframes
whole_economy_dynamism = calculate_dynamism_rates(whole_economy_df)
firm_size_dynamism = calculate_dynamism_rates(firm_size_df, group_by_cols=['emp_sizeband'])
firm_age_dynamism = calculate_dynamism_rates(firm_age_df, group_by_cols=['age_group'])
industry_dynamism = calculate_dynamism_rates(industry_df, group_by_cols=['industry_name'])
region_dynamism = calculate_dynamism_rates(region_df, group_by_cols=['region'])

In [78]:
# FIGURE 1 - BSD BASIC FACTS
# 4x4 facet of 
#   - number of firms
#   - total employees
#   - total turnover
#   - average turnover per employee (plus other deciles?)

# UPDATE WITH TURNOVER AND TURNOVER PER EMPLOYEE AFTER SECURELAB EXPORT

chart1 = alt.Chart(whole_economy_df).mark_line().encode(
    x=alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
    y=alt.Y('n_firms:Q',title='Total number of firms in BSD', scale=alt.Scale(domainMin=1500000, domainMax=2500000),axis=alt.Axis(format=".2s"))
)
chart1

chart2 = alt.Chart(whole_economy_df).mark_line().encode(
    x=alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
    y=alt.Y('employment:Q',title='Total employment in BSD', scale=alt.Scale(domainMin=15000000, domainMax=22000000), axis=alt.Axis(format=".2s"))
)
chart2

final_chart = chart1 | chart2

final_chart.save('Paper charts/fig1_bsd_basic_facts.png', scale_factor=2)
final_chart.save('Paper charts/fig1_bsd_basic_facts.json')

In [113]:
# FIGURE 2 - ECONOMIC CONTRIBUTION BY FIRM SIZE 2023
firmsize_2023 = firm_size_df[firm_size_df['year']==2023]

firmsize_2023['Total_employment'] = firmsize_2023['employment'].sum()
firmsize_2023['Total_firms'] = firmsize_2023['n_firms'].sum()

firmsize_2023['Share_of_employment'] = firmsize_2023['employment']/firmsize_2023['Total_employment']
firmsize_2023['Share_of_firms'] = firmsize_2023['n_firms']/firmsize_2023['Total_firms']

firmsize_share_of_activity = firmsize_2023.melt(id_vars='emp_sizeband',
                                                value_vars=['Share_of_employment','Share_of_firms'],
                                                value_name='Share of activity')

label_map = {
    'Share_of_employment': 'Employment',
    'Share_of_firms': 'Firms'
}

firmsize_share_of_activity['variable'] = firmsize_share_of_activity['variable'].map(label_map)

sizeband_order = ['Micro (0-9)', 'Small (10-49)', 'Medium (50-249)', 'Large (250+)']

chart = alt.Chart(firmsize_share_of_activity).mark_bar().encode(
    x=alt.X('emp_sizeband:O', sort=sizeband_order),
    y=alt.Y('Share of activity:Q', axis=alt.Axis(format='%')),
    color=alt.Color('variable:N').legend(title=None, orient='bottom', 
        direction='horizontal'),
    xOffset=alt.XOffset('variable:N')
)


chart.save('Paper charts/fig2_firmsize_share_of_activity.png', scale_factor=2)
chart.save('Paper charts/fig2_firmsize_share_of_activity.json')

/var/folders/dp/dmj0l1fj4cld5dkvtqpy9bqc0000gn/T/ipykernel_46369/2042026996.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  firmsize_2023['Total_employment'] = firmsize_2023['employment'].sum()
/var/folders/dp/dmj0l1fj4cld5dkvtqpy9bqc0000gn/T/ipykernel_46369/2042026996.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  firmsize_2023['Total_firms'] = firmsize_2023['n_firms'].sum()
/var/folders/dp/dmj0l1fj4cld5dkvtqpy9bqc0000gn/T/ipykernel_46369/2042026996.py:7: SettingWithCopyWarning: 
A value is trying

In [49]:
# ENTRY AND EXIT PLUS EMPLOYMENT WEIGHTED
entry_exit_rates = whole_economy_dynamism.melt(id_vars='year',value_vars=['Entry rate','Exit rate'])
emp_weighted_entry_exit_rates = whole_economy_dynamism.melt(id_vars='year',value_vars=['Entry job creation rate','Exit job destruction rate'])

entry_exit_rates['variable'] = entry_exit_rates['variable'].replace({
    'Entry rate': 'Entry', 
    'Exit rate': 'Exit'
})

emp_weighted_entry_exit_rates['variable'] = emp_weighted_entry_exit_rates['variable'].replace({
    'Entry job creation rate': 'Entry', 
    'Exit job destruction rate': 'Exit'
})

counts_based = alt.Chart(entry_exit_rates).mark_line().encode(
    x=alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
    y=alt.Y('value:Q', axis=alt.Axis(format='%'), title='Entry and exit rate (number of firms)'),
    color=alt.Color('variable:N', legend=alt.Legend(title=None, orient='bottom', direction='horizontal', labelFontSize=12,))
)

emp_weighted = alt.Chart(emp_weighted_entry_exit_rates).mark_line().encode(
    x=alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
    y=alt.Y('value:Q', axis=alt.Axis(format='%'), title='Entry and exit rate (employment weighted)'),
    color=alt.Color('variable:N', legend=alt.Legend(title=None, orient='bottom', direction='horizontal', labelFontSize=12,))
)

final_chart = counts_based | emp_weighted
final_chart

final_chart.save('Charts/entry_exit_emp_weighted.png', scale_factor=2)
final_chart.save('Charts/entry_exit_emp_weighted.json')

In [51]:
# Entry rates by industry

# Plot just entry rate
industry_entry_df = industry_dynamism.melt(id_vars=['year','industry_name'],
                                                 value_vars=['Entry rate'])

# Just display selected industries
# - IT
# - Recreation and culture
# - Other business
# - Retail
# - Transport
highlight_industries = ['IT', 'Recreation & Culture','Other business','Retail','Transport']

"#E6224B","#F4C245","#0063AF","#00A767" "#179FDB"

custom_color_map = {'IT':'#0063AF',
                    'Recreation & Culture':'#F4C245',
                    'Other business':'#E6224B',
                    'Retail':"#00A767",
                    'Transport':'#179FDB'}

custom_color_dict = dict(custom_color_map)

color_condition = alt.condition(
    alt.FieldOneOfPredicate(field='industry_name', oneOf=highlight_industries),
    alt.Color(
        'industry_name:N', 
        scale=alt.Scale(domain=list(custom_color_dict.keys()), range=list(custom_color_dict.values())),
        title=None),
        alt.value('grey')                         # Set all other lines to grey
)

opacity_condition = alt.condition(
    # TEST: Is the industry_name IN the list of highlighted industries?
    alt.FieldOneOfPredicate(field='industry_name', oneOf=highlight_industries),
    alt.value(1.0),   # VALUE_IF_TRUE: Full opacity (1.0)
    alt.value(0.1)    # VALUE_IF_FALSE: Reduced opacity (0.3)
)

chart = alt.Chart(industry_entry_df).mark_line().encode(
    x=alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
    y=alt.Y('value:Q', axis=alt.Axis(format='%'), title='Firm Entry Rate'),
    color=color_condition,
    opacity=opacity_condition,
    tooltip=['year:O','industry_name:N', 'value:Q']
)

chart

alt.Chart(...)

In [80]:
# Table of entry and exit rates by industry

bins = [1999, 2007, 2016, 2022]
labels = ['1998-2007', '2008-2016', '2017-2022']

# 3. Assign each year to a period
industry_dynamism['period'] = pd.cut(industry_dynamism['year'], bins=bins, labels=labels)

# 4. Create the pivot table (Rows = Industry, Columns = Period)
avg_entry_table = industry_dynamism.pivot_table(
    values='Entry rate', 
    index='industry_name', 
    columns='period', 
    aggfunc='mean'
)

avg_entry_table_final = (avg_entry_table * 100).round(1)
avg_entry_table_final

avg_entry_table_final.to_html('industry_entry_rates.html',index=False)

/var/folders/dp/dmj0l1fj4cld5dkvtqpy9bqc0000gn/T/ipykernel_46369/2494871611.py:10: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  avg_entry_table = industry_dynamism.pivot_table(


In [10]:
# FIGURE 4 - INDUSTRY ENTRY RATES 2018 TO 2022

# Note we could include 2023 here, currently blanked in the SecureLab export but theoretically we have the info we need.
# ENTRY = NOT PRESENT IN T-1 BUT PRESENT IN T (we don't need t+1)
# Check the figures in SecureLab

industry_entry = industry_dynamism.melt(id_vars=['year','industry_name'], value_vars=['Entry rate'])

industry_entry

chart = alt.Chart(industry_entry).mark_line().encode(
    x=alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
    y=alt.Y('value:Q',  axis=alt.Axis(format='%')),
    color=alt.Color('industry_name:N')
)

chart

alt.Chart(...)

In [ ]:
# FIGURE X - JOB CREATION

# Calculate 

job_creation = firm_size_dynamism.melt(id_vars=['year','emp_sizeband'],value_vars=['Entry job creation rate','Incumbent job creation rate'])

chart = alt.Chart(job_creation).mark_bar().encode(
    x=alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
    y=alt.Y('value:Q'),
    color=alt.Color('variable:N'),
    facet=alt.Facet('emp_sizeband:O', columns=2, sort=size_order, header=alt.Header(
            title='Job creation by firm size (employment)',
            titleFontSize=16,
            labelFontSize=12
        ))
)

job_creation = firm_size_dynamism.melt(id_vars=['year','emp_sizeband'],value_vars=['Incumbent job creation rate'])

chart = alt.Chart(job_creation).mark_bar().encode(
    x=alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
    y=alt.Y('value:Q'),
    color=alt.Color('emp_sizeband:N')
)
chart

alt.Chart(...)

In [54]:
# FIGURE X - JOB REALLOCATION
whole_economy_dynamism['Job reallocation'] = whole_economy_dynamism['jc_entrants'] + whole_economy_dynamism['jc_incumbents'] + whole_economy_dynamism['jd_exiters'] + whole_economy_dynamism['jd_incumbents']   
whole_economy_dynamism['Job reallocation rate'] = whole_economy_dynamism['Job reallocation'] / whole_economy_dynamism['total_employment_lagged']

whole_economy_dynamism

chart = alt.Chart(whole_economy_dynamism).mark_line().encode(
    x=alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
            y=alt.Y('Job reallocation rate:Q', axis=alt.Axis(format='%'), title='Job reallocation rate')).properties(
    width=600,  # Sets the interior width of the chart
    height=400  # Sets the interior height of the chart
)

trendline = chart.transform_regression('year', 'Job reallocation rate').mark_line(
    strokeDash=[2, 2], 
    color='grey',
    opacity=0.6
)
chart = chart + trendline
chart

chart.save('Paper charts/total_job_reallocation.png', scale_factor=2)
chart.save('Paper charts/total_job_reallocation.json')

In [40]:
# FIGURE X - SHARE OF EMPLOYMENT AND TURNOVER IN YOUNG FIRMS

# For this excercise, combine New and Young firms into a single 'Young (under 5 years)' category
age_map = {
    'New (0-2 years)': 'Young (under 5 years)',
    'Young (3-5 years)': 'Young (under 5 years)',
    'Mature (6-10 years)': 'Mature',
    'Old (11+ years)': 'Old'
}

firm_age_df['age_group_combined'] = firm_age_df['age_group'].map(age_map)

# Calculate total employment and turnover by year
total_by_year = firm_age_df.groupby('year').agg({
    'employment': 'sum',
    'n_firms':'sum'}).rename(columns={'employment': 'total_employment',
                                          'n_firms':'total_firms'})

young_firms = firm_age_df.merge(total_by_year, on='year')

young_firms = young_firms.groupby(['year','age_group_combined']).agg({'employment':'sum',
                                                                      'total_employment':'first',
                                                                      'n_firms':'sum',
                                                                      'total_firms':'first'}).reset_index()
young_firms = young_firms[young_firms['age_group_combined']=='Young (under 5 years)']
young_firms['Share_of_employment'] = young_firms['employment'] / young_firms['total_employment']
young_firms['Share_of_firms'] = young_firms['n_firms'] / young_firms['total_firms']

young_firms

young_firms = young_firms[young_firms['year']>=1999]

# Chart

chart1 = alt.Chart(young_firms).mark_line().encode(
    x=alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
            y=alt.Y('Share_of_employment:Q', axis=alt.Axis(format='%'), title='Share of employment in young firms')
)

trendline = chart1.transform_regression('year', 'Share_of_employment').mark_line(
    strokeDash=[2, 2], 
    color='grey',
    opacity=0.6
)
chart1 = chart1 + trendline

chart1

chart2 = alt.Chart(young_firms).mark_line().encode(
    x=alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
            y=alt.Y('Share_of_firms:Q', axis=alt.Axis(format='%'), title='Young firms as a share of total firms')
)

trendline = chart2.transform_regression('year', 'Share_of_firms').mark_line(
    strokeDash=[2, 2], 
    color='grey',
    opacity=0.6
)
chart2 = chart2 + trendline

final_chart = chart2 | chart1
final_chart

final_chart.save('Charts/young_firm_contribution.png', scale_factor=2)
final_chart.save('Charts/young_firm_contribution.json')

In [41]:
# FIGURE X - site expansion
# Didn't export total number of sites so can't do rates only absolute values
# How many sites are present each year?

site_expansion = whole_economy_dynamism.rename(columns={'site_exp_entrants':'New firms opening sites',
                                                  'site_exp_incumbents':'Existing firms opening new sites'})
site_expansion = site_expansion.melt(id_vars='year',value_vars=['New firms opening sites','Existing firms opening new sites'])

# Plot bar chart
chart = alt.Chart(site_expansion).mark_bar().encode(
    x = alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
    y = alt.Y('value:Q', title='Site expansion'),
    color=alt.Color('variable:N', legend=None)
)

# Add end labels
end_point = site_expansion.groupby(
    ['variable']
)['year'].idxmax()

end_point_data = site_expansion.loc[end_point]

text_labels = alt.Chart(end_point_data).mark_text(
    align='left',     
    dx=10,           
    baseline='middle'
).encode(
    x=alt.X('year:O'),         
    y=alt.Y('value:Q'),   
    text='variable:N',
    color=alt.Color('variable:N')
)

chart = chart + text_labels
chart

chart.save('Charts/site_expansion.png', scale_factor=2)
chart.save('Charts/site_expansion.json')

In [54]:
site_expansion_industry = industry_df.rename(columns={'site_exp_entrants':'New firms opening sites',
                                                  'site_exp_incumbents':'Existing firms opening new sites'})
site_expansion_industry = site_expansion_industry.melt(id_vars=['year','industry_name'],value_vars=['New firms opening sites','Existing firms opening new sites'])


chart = alt.Chart(site_expansion_industry).mark_bar().encode(
    x = alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
    y = alt.Y('value:Q', title='Site expansion'),
    color=alt.Color('variable:N'),
    facet=alt.Facet('industry_name:N', columns=3)
)

chart = chart.resolve_axis(
    x='independent',
)

chart

alt.Chart(...)

In [56]:
site_expansion_region = region_dynamism.rename(columns={'site_exp_entrants':'New firms opening sites',
                                                  'site_exp_incumbents':'Existing firms opening new sites'})
site_expansion_region = site_expansion_region.melt(id_vars=['year','region'],value_vars=['New firms opening sites','Existing firms opening new sites'])


chart = alt.Chart(site_expansion_region).mark_bar().encode(
    x = alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
    y = alt.Y('value:Q', title='Site expansion'),
    color=alt.Color('variable:N'),
    facet=alt.Facet('region:N', columns=3)
)

chart = chart.resolve_axis(
    x='independent',
)

chart

alt.Chart(...)

In [11]:
# FIGURE 3 - EXIT RATES

# Plot just exit rate
exit_df = whole_economy_dynamism.melt(id_vars=['year'],
                                                 value_vars=['Exit rate'])

chart = alt.Chart(exit_df).mark_line().encode(
    x=alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
    y=alt.Y('value:Q', axis=alt.Axis(format='%'), title='Market Sector Firm Exit Rate'),
    color=alt.Color('variable:N', title=None)

)

chart

alt.Chart(...)

In [56]:
# FIGURE 3 - EXIT RATES ACROSS FIRM SIZES

firmsize_exit_df = firm_size_dynamism.melt(id_vars=['year','emp_sizeband'],
                                                 value_vars=['Exit rate'])

chart = alt.Chart(firmsize_exit_df).mark_line().encode(
    x=alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
    y=alt.Y('value:Q', axis=alt.Axis(format='%'), title='Firm Exit Rate'),
    color=alt.Color('emp_sizeband:O', title=None, legend=None)

).properties(
    width=600,  # Sets the interior width of the chart
    height=400  # Sets the interior height of the chart
)


# Add end labels
end_point = firmsize_exit_df.groupby(
    ['emp_sizeband']
)['year'].idxmax()

end_point_data = firmsize_exit_df.loc[end_point]

end_point_data.loc[end_point_data['emp_sizeband'] == 'Large (250+)', 'value'] -= 0.003
end_point_data.loc[end_point_data['emp_sizeband'] == 'Medium (50-249)', 'value'] += 0.005

text_labels = alt.Chart(end_point_data).mark_text(
    align='left',     
    dx=10,           
    baseline='middle'
).encode(
    x=alt.X('year:O'),         
    y=alt.Y('value:Q'),   
    text='emp_sizeband:O',
    color=alt.Color('emp_sizeband:O')
)

chart = chart + text_labels
chart.save('Charts/exit_rate_firmsize.png', scale_factor=2)
chart.save('Charts/exit_rate_firmsize.json')

In [13]:
# JOB DESTRUCTION RATES

job_destruction_df = firm_size_dynamism.melt(id_vars=['year','emp_sizeband'],
                                                 value_vars=['Exit job destruction rate','Incumbent job destruction rate'])

chart = alt.Chart(job_destruction_df).mark_bar().encode(
    x=alt.X('year:O', axis=alt.Axis(
                labelExpr="datum.value % 2 == 0 ? datum.label : ''",  # Show every 2nd year
            labelAngle=0)),
    y=alt.Y('value:Q',axis=alt.Axis(format='%'), title='Job Destruction Rate'),
    color=alt.Color('variable:N', title=None),
    facet=alt.Facet('emp_sizeband:O', columns=2, sort=size_order, header=alt.Header(
            title='Job destruction by firm size (employment)')
))

chart

alt.Chart(...)